# 基于streamlit的大模型应用开发——问答与记忆

In [1]:
#导入语言模型
import os
from langchain_community.llms import Tongyi
from langchain_community.llms import SparkLLM
from langchain_community.llms import QianfanLLMEndpoint

import pandas as pd
#导入模版
from langchain.prompts import PromptTemplate

#导入聊天模型
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain_community.chat_models import ChatSparkLLM
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_community.chat_models import QianfanChatEndpoint

#输入三个模型各自的key

os.environ["DASHSCOPE_API_KEY"] = ""

os.environ["IFLYTEK_SPARK_APP_ID"] = ""
os.environ["IFLYTEK_SPARK_API_KEY"] = ""
os.environ["IFLYTEK_SPARK_API_SECRET"] = ""

os.environ["QIANFAN_AK"] = ""
os.environ["QIANFAN_SK"] = ""
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [2]:
model_ty = Tongyi(temperature=0)
model_qf = QianfanLLMEndpoint(model="ERNIE-Bot")
chat_qf = QianfanChatEndpoint(model="ERNIE-Bot")
chat_xh = ChatSparkLLM()
chat_ty= ChatTongyi()

In [3]:
import langchain
langchain.__version__

'0.1.16'

In [5]:
#! pip install --upgrade langchain -i https://mirrors.aliyun.com/pypi/simple

## langchain的大模型对话应用的实现


### 简单对话

In [5]:
model_ty = Tongyi(temperature=0)
model_ty.invoke("你好")

'你好！有什么我能为你做的吗？'

* streamlit的知识点

1. st.title()
2. st.write()
3. st.chat_input()
4. st.chat_message
5. python中冒号+等于号的作用

* 运行方法:打开终端——运行 streamlit run 文件地址

In [1]:
import streamlit as st

In [ ]:
#! pip install streamlit -i https://mirrors.aliyun.com/pypi/simple

In [6]:
# without walrus
n = 30
if n > 10:
    print(f"{n} is greater than 10")

# with walrus
if (n := 30) > 10:
    print(f"{n} is greater than 10")


30 is greater than 10
30 is greater than 10


* classwork 1

完成最简单大模型问答应用

### 带记忆对话

大多数的 LLM 应用程序都会有一个会话接口，允许我们和 LLM 进行多轮的对话，并有一定的上下文记忆能力。但实际上，模型本身是不会记忆任何上下文的，只能依靠用户本身的输入去产生输出。而实现这个记忆功能，就需要额外的模块去保存我们和模型对话的上下文信息，然后在下一次请求时，把所有的历史信息都输入给模型，让模型输出最终结果。

而在 LangChain 中，提供这个功能的模块就称为 Memory，用于存储用户和模型交互的历史信息。在 LangChain 中根据功能和返回值的不同，会有多种不同的 Memory 类型，主要可以分为以下几个类别：

* 对话缓冲区内存（ConversationBufferMemory），最基础的内存模块，用于存储历史的信息
* 对话缓冲器窗口内存（ConversationBufferWindowMemory），只保存最后的 K 轮对话的信息，因此这种内存空间使用会相对较少
* 对话摘要内存（ConversationSummaryMemory），这种模式会对历史的所有信息进行抽取，生成摘要信息，然后将摘要信息作为历史信息进行保存。
* 对话摘要缓存内存（ConversationSummaryBufferMemory），这个和上面的作用基本一致，但是有最大 token 数的限制，达到这个最大 token 数的时候就会进行合并历史信息生成摘要

值得注意的是，对话摘要内存的设计出发点就是语言模型能支持的上下文长度是有限的（一般是 2048），超过了这个长度的数据天然的就被截断了。这个类会根据对话的轮次进行合并，默认值是 2，也就是每 2 轮就开启一次调用 LLM 去合并历史信息。

In [7]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

a1=ConversationBufferMemory()

conversation = ConversationChain(
    llm=model_ty,  
    memory=a1
)

In [8]:
conversation.invoke("你好")

{'input': '你好',
 'history': '',
 'response': '你好！有什么问题或者需要帮助的吗？我可以提供各种信息，从天气预报到科技新闻，从食谱建议到历史事件的详情。只要告诉我你感兴趣的话题，我们就可以聊起来！'}

In [9]:
conversation.memory.chat_memory.messages

[HumanMessage(content='你好'),
 AIMessage(content='你好！有什么问题或者需要帮助的吗？我可以提供各种信息，从天气预报到科技新闻，从食谱建议到历史事件的详情。只要告诉我你感兴趣的话题，我们就可以聊起来！')]

In [10]:
a1.chat_memory.messages[0].type

'human'

In [11]:
a1.chat_memory.messages

[HumanMessage(content='你好'),
 AIMessage(content='你好！有什么问题或者需要帮助的吗？我可以提供各种信息，从天气预报到科技新闻，从食谱建议到历史事件的详情。只要告诉我你感兴趣的话题，我们就可以聊起来！')]

* streamlit的会话状态与回调方式(session_state)

In [ ]:
if 'history' not in st.session_state:
    st.session_state['history'] = []

* classwork 2

完成带记忆的大模型问答应用

## 基于streamlit的大模型问答应用实现-通义千问

* 原生通义千问问答实现

In [12]:
import dashscope
from dashscope import Generation
from langchain.tools.render import render_text_description
import streamlit as st

dashscope.api_key = ""

In [13]:
prompt=[]

def get_response(mess):
    response = Generation.call(
        model='qwen-turbo',
        messages=mess,
        result_format='message', # 将输出设置为message形式
    )
    return response

def prompt_yun(ct):
    prompt.append({"role":"user","content":ct})
    return prompt

In [14]:
p_y=prompt_yun("你好")
res=get_response(p_y)

In [15]:
res.output.choices[0].message['content']

'你好！很高兴能为你服务。有什么可以帮助你的吗？'

## 带记忆的对话问答的streamlit实现-通义千问

* 原生通义千问的记忆方案

* classwork 3 

1. 完成通义千问问答简单搭建
2. 完成带记忆的通义千问问答应用的搭建

* 应用打包

https://blog.csdn.net/Helloorld_1/article/details/131767478

https://blog.csdn.net/nexttoparadise/article/details/135368387

* langchain官方文档对话

https://chat.langchain.com/